In [32]:
# !pip install -r requirement.txt

In [1]:
import pandas as pd 
import os 
import numpy as np 
from sklearn.cluster import KMeans
from shapely.geometry import Polygon, Point
from sklearn.model_selection import KFold
from sklearn.metrics import  mean_squared_error
from sklearn.decomposition import PCA
from os.path import join
import catboost as cat
import shapefile as shp
from tqdm import tqdm
from vincenty import vincenty
import requests
import time

## utils

In [2]:
def read_shapefile(path, num):

    sh = shp.Reader(join(path, 'zaf_admbnda_adm{}_2016SADB_OCHA.shp'.format(num)))
    print(len(sh.shapes()))

    shapeRecs = sh.shapeRecords()
    all_shapes = sh.shapes()
    data_ = []
    shapes = []
    for i in range(len(sh.shapes())-1):
        data_.append(shapeRecs[i].record)
        shapes.append(all_shapes[i].points)
    df = pd.DataFrame.from_records(data_)
    fields = sh.fields
    df.columns =[f[0] for f in fields[1:]]
    df['shape_adm{}'.format(num)] = shapes
    df['shape_adm{}'.format(num)] = df['shape_adm{}'.format(num)].apply(lambda x:Polygon(x))
    df['area_adm{}'.format(num)] = df['shape_adm{}'.format(num)].apply(lambda x:x.area)
    df['centroid_lon{}'.format(num)] = df['shape_adm{}'.format(num)].apply(lambda x:(x.centroid).coords[0][0])
    df['centroid_lat{}'.format(num)] = df['shape_adm{}'.format(num)].apply(lambda x:(x.centroid).coords[0][1])
    return df

In [3]:
train=pd.read_csv("./Train.csv")
test=pd.read_csv("./Test.csv")

In [4]:
data=pd.concat([train,test],ignore_index=True)
max_1=data["total_households"].max()
max_2=data["total_individuals"].max()
data["total_households"]/=max_1
data["total_individuals"]/=max_2

In [5]:
# disposal of luxury items
luxury_stuff = ['psa_01','car_01','stv_00']
not_luxury_stuff = ['psa_00','car_00','stv_01']
data['luxury_stuff'] = data[luxury_stuff].sum(axis=1)
data['not_luxury_stuff'] = data[not_luxury_stuff].sum(axis=1)

In [6]:
# Kmean clusters
to_drop=["dw_12","dw_13","lan_13","pw_07","pw_08",'dw_00','dw_02', 'dw_06','psa_02','lan_02','lan_03','lan_04','lan_05','lan_08', 'pw_01' , 'lan_07','NL']
data_copy=data.copy()
columns=data_copy.drop(["ward","ADM4_PCODE","target"]+to_drop,1).columns
data_copy=data_copy[columns]
km=KMeans(10,random_state=1992)
km=km.fit(data_copy[columns])
# predict the cluster of each area
data["cluster"]=km.predict(data_copy[columns]) 

In [7]:
# reduce dimentionality for dwelling features
pca = PCA()
dwelling_features =  data.filter(regex='dw_.*')
df_pca = pca.fit_transform(dwelling_features)

data['pca_dw_0'] = df_pca[:,0]
data['pca_dw_1'] = df_pca[:,1]
# reduce dimentionality for language features
pca = PCA()
lan_features =  data.filter(regex='lan_.*')
df_pca = pca.fit_transform(lan_features)

data['pca_lan_0'] = df_pca[:,0]
data['pca_lan_1'] = df_pca[:,1]

In [8]:
# calculate the household size
data['Household_Size'] = (data['total_individuals']/data['total_households'])

In [9]:
# get the most frequent dwelling type
arank = data.filter(regex='dw_.*').apply(np.argsort, axis=1)
ranked_lang = pd.DataFrame(data.filter(regex='dw_.*').columns.to_series()[arank.values[:,::-1][:,1:4]])
data['dwelling_type_1'] = ranked_lang[0]
data['dwelling_type_2'] = ranked_lang[1]
data['dwelling_type_3'] = ranked_lang[2]


data.loc[data.dwelling_type_1.isin(['dw_07','dw_08']), 'dwelling_type_1'] = 'shack'
data.loc[data.dwelling_type_2.isin(['dw_07','dw_08']), 'dwelling_type_2'] = 'shack'
data.loc[data.dwelling_type_3.isin(['dw_07','dw_08']), 'dwelling_type_3'] = 'shack'


luxury_dwelling_type = ['dw_02', 'dw_03', 'dw_04', 'dw_05', 'dw_06', 'dw_09']
data.loc[data.dwelling_type_1.isin(luxury_dwelling_type), 'dwelling_type_1'] = 'luxury'
data.loc[data.dwelling_type_2.isin(luxury_dwelling_type), 'dwelling_type_2'] = 'luxury'
data.loc[data.dwelling_type_3.isin(luxury_dwelling_type), 'dwelling_type_3'] = 'luxury'
data['dw_luxury'] = data[luxury_dwelling_type[0:6]].sum(axis=1)

In [10]:
#water access
data['water_access'] = data.filter(regex='pw_.*').idxmax(axis=1)

In [11]:
# get the top points of interests(POIs) of each ADM in specific categories such as 'Facilities', 'Education Facility', etc

BASE_URL = "https://places.sit.ls.hereapi.com/places/v1/"
NEARBY_URL = BASE_URL + "discover/here?at={},{}&apiKey={}"
SUGGEST_URL = BASE_URL + "/autosuggest?in={},{};r=300000&q={}'&apiKey={}"

categories = ['hospital or health care facility','Facilities', 'Education Facility','Public Transport',\
 'Government or Community Facility']

list_dicts = []
for i in tqdm(range(data.shape[0])):
    for cat in categories:
        resp = requests.get(url=SUGGEST_URL.format(data.loc[i,'lat'], data.loc[i,'lon'], cat,"h5lCzAFIoFjRPp94SKO1uTIjjcIkaR3h_uWduXUvcRI" ))
        try:
            resp = resp.json()
            resp['id'] = i
            resp['suggested_type'] = cat
            list_dicts.append(resp)
        except: 
            print(i)

In [12]:
# calculate mean distance to each category type
for list_ in tqdm(list_dicts): 
    id_ = list_['id']
    try:
        results = list_['results']
        cat = results[0]['category']
        data.loc[id_, 'nb_establishments_type_{}'.format(cat)] = len(results)
        distances = []
        for i in range(len(results)):
            if 'distance' in results[i].keys():
                distances.append(results[i]['distance'])
        data.loc[id_, 'mean_distance_to_establishments_type_{}'.format(cat)] = np.mean(distances)
    except:
        pass

In [11]:
df_adm4 = read_shapefile('./zaf_adm_2016SADB_OCHA_SHP/',4)
df_adm3 = read_shapefile('./zaf_adm_2016SADB_OCHA_SHP/',3)
df_adm2 = read_shapefile('./zaf_adm_2016SADB_OCHA_SHP/',2)

4392
213
52


In [12]:
df_adm4 = df_adm4.merge(df_adm3[['ADM3_PCODE', 'area_adm3','centroid_lon3', 'centroid_lat3']], on=['ADM3_PCODE'], how='left')
df_adm4 = df_adm4.merge(df_adm2[['ADM2_PCODE', 'area_adm2','centroid_lon2', 'centroid_lat2']], on=['ADM2_PCODE'], how='left')
df = data.copy()
df = df.merge(df_adm4.filter(regex='(area.*)|(ADM4_PCODE)|(centroid.*)'), on=['ADM4_PCODE'], how='left')
assert(df.shape[0]==data.shape[0])

In [13]:
data = df.copy()

In [14]:
#get ID of ADM4_PCODE
data["ADM4_PCODE"]=data["ADM4_PCODE"].str[2:].astype(int)

In [15]:
# distance to center of each ADM
data["distance_to_center"] = data.apply(
    lambda x: vincenty((x["lat"], x["lon"]), (x["centroid_lat3"], x["centroid_lon3"]),'kilometers'), axis = 1)

data["distance_to_center2"] = data.apply(
    lambda x: vincenty((x["lat"], x["lon"]), (x["centroid_lat2"], x["centroid_lon2"]),'kilometers'), axis = 1)

In [16]:
data["total_households"]*=max_1
data["total_individuals"]*=max_2

In [17]:
train=data[~data.target.isna()]
test=data[data.target.isna()]

In [18]:
if not os.path.exists('./submissions/'):
    os.mkdir('./submissions/')
i = len(os.listdir('./submissions'))+1
def train_func(train,test,target_name,features,params,num_class,submission_name='sub_'.format(i)):
    oof_train = np.zeros((len(train),num_class))
    oof_test  = np.zeros((len(test),num_class))
    importances=[]
    cv_scores = []
    train_scores=[]
    cat_features = []
    dtest=cat.Pool(data=test[features], cat_features=cat_features, feature_names=features)
    
    for ind, (trn_ind, val_ind) in ( enumerate(kfolds.split(train,train[target_name])) ):

        print('fold_n',ind+1, 'started at', time.ctime())
        X_train, X_valid = train.loc[trn_ind][features], train.loc[val_ind][features]
        y_train, y_valid = train.loc[trn_ind][target_name], train.loc[val_ind][target_name]
        dtrain=cat.Pool(data=X_train[features],label=y_train,cat_features=cat_features,
                        feature_names=features)
        dval=cat.Pool(data=X_valid[features],label=y_valid,cat_features=cat_features,
                        feature_names=features)
 
        model = cat.CatBoost(params)
    
        model.fit(dtrain, eval_set=dval, use_best_model=True)

        val_pred = model.predict(dval)
        train_pred = model.predict(dtrain,thread_count=4)
        test_pred = model.predict(dtest,thread_count=4)

              
        oof_train[val_ind,:] += np.reshape(val_pred,(-1,num_class))
        oof_test += np.reshape(test_pred,(-1,num_class))
        score_fold_validation=np.sqrt(mean_squared_error(y_valid, val_pred))
        score_fold_train=np.sqrt(mean_squared_error(y_train, train_pred))

        train_scores.append(score_fold_train)
        cv_scores.append(score_fold_validation)
        print('-Train Score: {} - CV Score : {}'.format(score_fold_train, score_fold_validation))
    
    end_train_score=np.mean(train_scores)
    train_scores.append(end_train_score)
    
    oof_score=np.sqrt(mean_squared_error(train[target_name], oof_train))
    cv_scores.append(oof_score)
    print("\n\ntraining is done : train score {} - oof Score {}".format(str(end_train_score),str(oof_score)))
    # create and save the submission
    submission=test[["ward"]]
    submission["target"]=oof_test/nfolds
    # check if the submission folder exists. If not, create it.
    submission.to_csv("./submissions/{}".format(submission_name),index=False)
    

In [19]:
train.reset_index(drop=True,inplace=True)
num_boost_round=10000000
nfolds = 5
kfolds = KFold(n_splits=nfolds, shuffle=True, random_state=15468)

In [20]:
features = ['ADM4_PCODE', 'car_00', 'car_01', 'dw_01', 'dw_03', 'dw_04', 'dw_05', 'dw_07', 'dw_08', 'dw_09', 'dw_10', 'dw_11', 'lan_00', 'lan_01', 'lan_06', 'lan_09', 'lan_10', 'lan_11', 'lan_12', 'lan_14', 'lat', 'lgt_00', 'lln_00', 'lln_01', 'lon', 'pg_00', 'pg_01', 'pg_02', 'pg_03', 'pg_04', 'psa_00', 'psa_01', 'psa_03', 'psa_04', 'pw_00', 'pw_02', 'pw_03', 'pw_04', 'pw_05', 'pw_06', 'stv_00', 'stv_01', 'total_households', 'total_individuals', 'luxury_stuff', 'not_luxury_stuff', 'cluster','area_adm4']
print(len(features))

48


In [21]:
# Train Model and predictions
params = {
    "learning_rate": 0.055,
    "random_seed": 0,
    
    "thread_count": -1,
    "iterations": 10000000,
    
    "loss_function": "RMSE",
    "eval_metric": "RMSE",
        
    "l2_leaf_reg": 3,
    "bagging_temperature": 1,  
    
    "depth": 4,
    "od_type": "Iter",
    "od_wait": 55,

    "verbose_eval": False,
    "use_best_model": True,
}
train_func(train,test,"target",features,params=params,num_class=1, submission_name='catboost_surface.csv')


fold_n 1 started at Fri Mar  6 11:03:39 2020
-Train Score: 1.8773650748270836 - CV Score : 3.283099492709999
fold_n 2 started at Fri Mar  6 11:03:41 2020
-Train Score: 1.4829423798019148 - CV Score : 3.1140304727040062
fold_n 3 started at Fri Mar  6 11:03:44 2020
-Train Score: 1.8306926360645501 - CV Score : 3.400646793117392
fold_n 4 started at Fri Mar  6 11:03:47 2020
-Train Score: 1.304331734453534 - CV Score : 2.9579554665491052
fold_n 5 started at Fri Mar  6 11:03:51 2020
-Train Score: 1.7122022944976076 - CV Score : 3.2045925537246283


training is done : train score 1.6415068239289379 - oof Score 3.195601138969688
